In [ ]:
!wget https://www.dropbox.com/s/241p0ooxybbkzpq/archive.zip?dl=0

--2023-05-16 06:48:46--  https://www.dropbox.com/s/241p0ooxybbkzpq/archive.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/241p0ooxybbkzpq/archive.zip [following]
--2023-05-16 06:48:46--  https://www.dropbox.com/s/raw/241p0ooxybbkzpq/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbebeab45e1f63a815fdf5b4c96.dl.dropboxusercontent.com/cd/0/inline/B8Kg1OL8AwMmmnaPH4BlgpQOjgETzzZ1RIzIp1uUVlBcG3EN5jbZs9yKVweCcE19nAebWQp-bPESAYdZxD3czWw56j3UfMUo0VrpnFWbl2AD612R6xBpEZ48Ya8En1TZtCA242teoXoeqv1DI8JmgoizIDFPvd22hmwcvSF0inm3lw/file# [following]
--2023-05-16 06:48:47--  https://ucbebeab45e1f63a815fdf5b4c96.dl.dropboxusercontent.com/cd/0/inline/B8Kg1OL8AwMmmnaPH4BlgpQOjgETzzZ1RIzIp1uUVlBcG3EN5jbZs9yKVweCcE19nAebWQp-bPE

In [ ]:
!unzip archive.zip\?dl\=0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
len(os.listdir("/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"))

In [ ]:
import keras

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg19 import VGG19 , preprocess_input , decode_predictions

In [ ]:
train_datagen = ImageDataGenerator(zoom_range= 0.5 ,shear_range = 0.3 ,horizontal_flip= True,preprocessing_function = preprocess_input )
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input )

In [ ]:
train = train_datagen.flow_from_directory(directory = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",target_size = (256,256),batch_size=32)

val = val_datagen.flow_from_directory(directory = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",target_size = (256,256),batch_size=32)



In [ ]:
t_img,label = train.next ()

In [ ]:
t_img.shape

In [ ]:
def plotImage(img_arr , label ):

  for im , l in zip(img_arr, label):
    plt.figure (figsize=(5,5))
    plt.imshow(im)
    plt.show()


In [ ]:
plotImage(t_img[:3], label[:3])

building our model

In [ ]:
from keras.layers import Dense, Flatten 
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras

In [ ]:
base_model = VGG19(input_shape = (256,256,3), include_top = False )


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
X = Flatten()(base_model.output)
X = Dense (units = 38,activation = 'softmax')(X)

#creating our model
model = Model(base_model.input,X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss= keras.losses.categorical_crossentropy,metrics=['accuracy'])

early stopping and model check point

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
#early stopping
es= EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01 , patience = 3, verbose = 1)
#Model Checkpoint
mc = ModelCheckpoint ( filepath=" best_model.h5",
                      monitor='val_accuracy',
                      min_deelta=0.01,
                      patience=2,
                      verbose=1,
                      save_best_only= True)
cb = [ es, mc]



In [ ]:
his = model.fit_generator(train ,
                          steps_per_epoch=16,
                          epochs=50,
                          verbose=1,
                          callbacks=cb,
                          validation_data=val,
                          validation_steps=16 )

In [ ]:
h= his.history
h.keys()


In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = "red")

plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "red")

plt.title("loss vs v-loss")
plt.show()

In [ ]:
#load best model
from keras.models import load_model
model= load_model("/content/ best_model.h5")

In [ ]:
acc = model.evaluate_generator(val)[1]
print(f"The accuracy of your model is = {acc*100} %")